# BEGIN

In [1]:
import sys
import os
from pathlib import Path
path_project = Path(os.getcwd()).parent # /home/o.lipina/workspace/rucubrik
path_src = path_project / 'src'  # /home/o.lipina/workspace/rucubrik/src
sys.path.append(str(path_src))
sys.path.append(str(path_project))

%load_ext autoreload
%autoreload 2

In [2]:
import requests
from datetime import datetime
import json
import logging
import json
# import pandas as pd
import pickle
from pathlib import Path
import re
import random
import time
from tqdm import tqdm
from urllib.parse import urlparse
import warnings
warnings.simplefilter("ignore")
# import numpy as np

random.seed(42)
# np.random.seed(42)

In [3]:
from src.config import CONFIG

In [4]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(levelname)s] %(name)s - %(message)s"
)

## Click tool

In [5]:
from src.tools.browser.environment import BrowserEnvironment
from src.tools.browser.get_text import GetTextTool
from src.tools.browser.click import ClickTool
from src.tools.browser.click import FillTool
from src.tools.browser.click import CheckContentTool
from src.tools.browser.dropdown import DropDownTool
from src.tools.browser.dropdown import DropdownOptionsTool


[INFO] config - Configuration loaded successfully from /home/olipina/workspace/org-agent/config.yaml


In [6]:
CONFIG.isu_booking_creds.booking_login

'https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688'

In [7]:
browser_env = BrowserEnvironment()
await browser_env.initialize()

[INFO] src.tools.browser.environment - Browser environment initialized


### Login

In [8]:
await browser_env.navigate(CONFIG.isu_booking_creds.booking_login)

In [9]:
browser_env.current_url

'https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688'

In [10]:
result = await FillTool().execute(
        env=browser_env,
        selector='#username',
        value=CONFIG.isu_booking_creds.username
        )
result = await FillTool().execute(
        env=browser_env,
        selector='#password',
        value=CONFIG.isu_booking_creds.password
        )

[INFO] src.tools.browser.click - Attempting to fill '311408' into input #username
[INFO] src.tools.browser.click - Successfully filled '311408' into #username
[INFO] src.tools.browser.click - Attempting to fill 'musummer1' into input #password


[INFO] src.tools.browser.click - Successfully filled 'musummer1' into #password


In [11]:
result = await ClickTool().execute(
        env=browser_env,
        text="Вход",
        
    )

[INFO] src.tools.browser.click - Attempting to click button with text: Вход


[INFO] src.tools.browser.click - Waiting for navigation to complete
[INFO] src.tools.browser.click - Click successful on text='Вход'. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2143:1:112030779307075


In [12]:
texts=["Личный кабинет", "Центр приложений"]
login_result = await CheckContentTool().execute(
    env=browser_env,texts=["Личный кабинет", "Центр приложений"])

[INFO] src.tools.browser.click - Found texts: ['Личный кабинет', 'Центр приложений']


### Go to booking

In [86]:
await browser_env.navigate(CONFIG.isu_booking_creds.booking_url)

In [87]:
browser_env.current_url

'https://isu.ifmo.ru/pls/apex/f?p=2431:1:112030779307075:'

In [88]:
result = await ClickTool().execute(
        env=browser_env,
        text="Создать заявку",
    )

[INFO] src.tools.browser.click - Attempting to click button with text: Создать заявку


[INFO] src.tools.browser.click - Waiting for navigation to complete
[INFO] src.tools.browser.click - Click successful on text='Создать заявку'. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


In [89]:
expected_texts = [
                "Карточка заявки",
                "Заявка",
                "Картотека заявок",
                "Проект заявки"
            ]
result = await CheckContentTool().execute(
    env=browser_env,texts=expected_texts)


[INFO] src.tools.browser.click - Found texts: ['Карточка заявки', 'Заявка', 'Картотека заявок', 'Проект заявки']


### Specify building

In [90]:
result = await DropdownOptionsTool().execute(
    env=browser_env,
    dropdown_selector = '.select2-chosen',
)

[INFO] src.tools.browser.dropdown - Opening dropdown by selector: .select2-chosen


[INFO] src.tools.browser.dropdown - Retrieving dropdown options
[INFO] src.tools.browser.dropdown - Closing dropdown with Escape key
[INFO] src.tools.browser.dropdown - Successfully retrieved 21 options


In [91]:
result.meta['options']

['Коворкинги и пространства',
 'ITMO PLACE (Ломоносова 9)',
 'Kronbars Space (Ломоносова 9)',
 'Коворкинг 1300 (Кронверкский 49)',
 'Тихий коворкинг в аудитории 1302',
 'Коворкинг Ломоносова 1303 (Ломоносова 9)',
 'Коворкинг Студенческого офиса 1400 (Ломоносова 9)',
 'Музей истории Университета ИТМО (Гривцова 14-16)',
 'Портал в Яндекс (Кронверкский 49)',
 'Общий аудиторный фонд',
 'Аудиторный фонд ИТМО (Гривцова 14-16)',
 'Аудиторный фонд ИТМО (Кронверкский 49)',
 'Аудиторный фонд ИТМО (Ломоносова 9)',
 'Аудиторный фонд ИТМО (Чайковского 11/2)',
 'Актовые залы',
 'Актовый зал (Гривцова 14-16)',
 'Актовый зал (Ломоносова 9)',
 'Конференц-залы',
 'Овальный зал пер (Гривцова 14-16)',
 'Виртуальные аудитории',
 'Виртуальные аудитории zoom']

In [92]:
expected_building = CONFIG.default_booking_params.building
expected_building

'Аудиторный фонд ИТМО (Кронверкский 49)'

In [93]:
result1 = await DropDownTool().execute(
        env=browser_env,
        option_text=expected_building,
        dropdown_selector='.select2-chosen',
    )

[INFO] src.tools.browser.dropdown - Clicking dropdown by selector: .select2-chosen


[INFO] src.tools.browser.dropdown - Selecting option: Аудиторный фонд ИТМО (Кронверкский 49)
[INFO] src.tools.browser.dropdown - Successfully selected 'Аудиторный фонд ИТМО (Кронверкский 49)' from dropdown


In [94]:
expected_texts = [expected_building]
result = await CheckContentTool().execute(
    env=browser_env,texts=expected_texts)


[INFO] src.tools.browser.click - Found texts: ['Аудиторный фонд ИТМО (Кронверкский 49)']


### Select date

In [95]:
date_str = "18-01-2025"
target_date = datetime.strptime(date_str, "%d-%m-%Y")
target_date_dots_str = datetime.strptime(date_str, "%d-%m-%Y").strftime("%d.%m.%Y")
target_date_dots_str

'18.01.2025'

In [96]:
res = ClickTool()
click_result = await ClickTool().execute(
            env=browser_env,
            selector="#P4_DATE",
            wait_for_navigation=False,
)

[INFO] src.tools.browser.click - Attempting to click element by selector: #P4_DATE
[INFO] src.tools.browser.click - Click successful on #P4_DATE. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


In [97]:
calendar_check = await CheckContentTool().execute(
                env=browser_env,
                texts=["ui-datepicker-calendar", "ui-datepicker-month", "ui-datepicker-year"]
            )

[INFO] src.tools.browser.click - Found texts: ['ui-datepicker-calendar', 'ui-datepicker-month', 'ui-datepicker-year']


In [98]:
# Get current month and year text
month_result = await GetTextTool().execute(
   env=browser_env,
   selector=".ui-datepicker-month"
)
year_result = await GetTextTool().execute( 
   env=browser_env,
   selector=".ui-datepicker-year"
)

if not month_result.success or not year_result.success:
#    logger.error("Failed to get current month/year")
#    return False
     print("Failed to get current month/year")

[INFO] src.tools.browser.get_text - Getting text by selector: .ui-datepicker-month
[INFO] src.tools.browser.get_text - Got text content: Январь
[INFO] src.tools.browser.get_text - Getting text by selector: .ui-datepicker-year
[INFO] src.tools.browser.get_text - Got text content: 2025


In [99]:
# how many time to click to get to the target month
current_month_text = month_result.meta['result']
current_year_text = year_result.meta['result']

month_map = {
    'Январь': 0, 'Февраль': 1, 'Март': 2, 'Апрель': 3,
    'Май': 4, 'Июнь': 5, 'Июль': 6, 'Август': 7,
    'Сентябрь': 8, 'Октябрь': 9, 'Ноябрь': 10, 'Декабрь': 11
}
current_month_num = month_map[current_month_text]
current_year_num = int(current_year_text)
target_month_idx = target_date.month - 1  # Convert to 0-based
target_year = target_date.year

months_diff = (target_year - current_year_num) * 12 + (target_month_idx - current_month_num)
months_diff

0

In [100]:
if months_diff > 0:
    for _ in range(months_diff):
        await ClickTool().execute(
            env=browser_env,
            selector=".ui-datepicker-next",  # Next month button
            wait_for_navigation=False,
        )

In [101]:
date_selector = (
    f'[data-handler="selectDay"]'
    f'[data-month="{target_date.month - 1}"]'  # jQuery UI months are 0-based
    f'[data-year="{target_date.year}"]'
    f' a:text("{target_date.day}")'
)

date_click = await ClickTool().execute(
    env=browser_env,
    selector=date_selector,
    wait_for_navigation=False,
)

[INFO] src.tools.browser.click - Attempting to click element by selector: [data-handler="selectDay"][data-month="0"][data-year="2025"] a:text("18")
[INFO] src.tools.browser.click - Click successful on [data-handler="selectDay"][data-month="0"][data-year="2025"] a:text("18"). URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


In [102]:
input_value = await browser_env.page.input_value("#P4_DATE")
input_value

'18.01.2025'

In [103]:
calendar_check = await CheckContentTool().execute(
                env=browser_env,
                texts=[target_date_dots_str]
            )

[INFO] src.tools.browser.click - Found texts: ['18.01.2025']


## Бронь аудиторий

### Определение нужной аудитории

In [104]:
from src.tools.booking_utils import GetAvailableRoomsTool, GetTimeIntervalsTool, GetRoomIdTool, CheckRoomAvailableTool
from src.scenarios.booking.booking import BookingParams

In [105]:
# открыть табличку выбора аудиторий

In [106]:
room_number = "1229"
booking_params = BookingParams(room_count=3)
start_time = booking_params.start_time
end_time = booking_params.end_time

In [107]:
available_rooms = await GetAvailableRoomsTool().execute(env= browser_env)
available_rooms

[INFO] src.tools.booking_utils - Found 21 available rooms


['1229',
 '1316',
 '1404',
 '1405',
 '1410',
 '1419',
 '1428',
 '2201',
 '2316',
 '2326',
 '2330',
 '2336',
 '2337',
 '2403',
 '2407',
 '2412',
 '2414',
 '2416',
 '2426',
 '2430',
 '2433']

In [108]:
room_number in available_rooms

True

In [109]:
# getting time intervals compatible with html table
time_intervals = GetTimeIntervalsTool().execute(start_time, end_time)
time_intervals

[INFO] src.tools.booking_utils - Time 19:00:00-22:50:00 overlaps with interval 7 (18:40:00-20:10:00)
[INFO] src.tools.booking_utils - Time 19:00:00-22:50:00 overlaps with interval 8 (20:20:00-21:50:00)
[INFO] src.tools.booking_utils - Time 19:00:00-22:50:00 overlaps with interval 9 (22:00:00-23:00:00)


['7', '8', '9']

In [110]:
room_id = await GetRoomIdTool().execute(
    env = browser_env,
    room_number = room_number,
)

[INFO] src.tools.booking_utils - Found room_id 145 for room number 1229


In [111]:
available = await CheckRoomAvailableTool().execute(
    env = browser_env,
    room_id = room_id,
    time_intervals = time_intervals,
    room_number = room_number
)
available

[INFO] src.tools.booking_utils - Room 1229 (ID=145) is available at interval 7
[INFO] src.tools.booking_utils - Room 1229 (ID=145) is available at interval 8
[WARNING] src.tools.booking_utils - Interval 9 is 22-23 and considered always available


[True, True, True]

In [112]:
# click on select id=P4_AUD_NAME
click_result = await ClickTool().execute(
            env=browser_env,
            selector="#P4_AUD_NAME",
            wait_for_navigation=False,
)

[INFO] src.tools.browser.click - Attempting to click element by selector: #P4_AUD_NAME
[INFO] src.tools.browser.click - Click successful on #P4_AUD_NAME. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


NameError: name 'P4_AUD_NAME' is not defined

In [113]:
cell = f'td.reserve[roomid="{room_id}"][interval="{time_intervals[0]}"]'
# click on cell
click_result = await ClickTool().execute(
            env=browser_env,
            selector=cell,
            wait_for_navigation=False,
)
click_result

[INFO] src.tools.browser.click - Attempting to click element by selector: td.reserve[roomid="145"][interval="7"]
[INFO] src.tools.browser.click - Click successful on td.reserve[roomid="145"][interval="7"]. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


ToolResponse(success=True, error=None, meta={'action': 'click', 'target': 'td.reserve[roomid="145"][interval="7"]', 'url': 'https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,', 'narrative': ['Attempting to click element by selector: td.reserve[roomid="145"][interval="7"]', 'Click successful on td.reserve[roomid="145"][interval="7"]. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,'], 'url_after_click': 'https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,'})

In [115]:
texts = [
    "Требуется техническое сопровождение мероприятия",
    "Контактный телефон",
    "Название мероприятия"
]
check = await CheckContentTool().execute(
                env=browser_env,
                texts=texts
            )
check

[INFO] src.tools.browser.click - Found texts: ['Требуется техническое сопровождение мероприятия', 'Контактный телефон', 'Название мероприятия']


ToolResponse(success=True, error=None, meta={'action': 'check_content', 'url': 'https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,', 'texts': ['Требуется техническое сопровождение мероприятия', 'Контактный телефон', 'Название мероприятия'], 'narrative': ["Found texts: ['Требуется техническое сопровождение мероприятия', 'Контактный телефон', 'Название мероприятия']"], 'found_texts': ['Требуется техническое сопровождение мероприятия', 'Контактный телефон', 'Название мероприятия'], 'missing_texts': []})

### А если она забронирована?

In [84]:
await CheckContentTool().execute(
                env=browser_env,
                texts=["scheduleTable"]
            )

[INFO] src.tools.browser_2.click - Found texts: ['scheduleTable']


BrowserToolResponse(success=True, error=None, meta={'action': 'check_content', 'url': 'https://isu.ifmo.ru/pls/apex/f?p=2431:4:117559366568926::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,', 'texts': 'scheduleTable', 'narrative': "Found texts: ['scheduleTable']", 'found_texts': 'scheduleTable', 'missing_texts': ''})

In [ ]:
initial_room_number = "2403"
interval_human_readable = UNIVERSITY_INTERVALS[interval]
...
room_id = "59"
interval = "2"
# interval = "7"
cell_selector = (
                f'td.reserve[roomid="{room_id}"][interval="{interval}"]'
            )
cell = await browser_env.page.query_selector(cell_selector)
logger.info("Success – cell found. Room %s is free at interval %s", initial_room_number, interval_human_readable)
if cell is None:
    cell_selector = (
                f'td.busy[roomid="{room_id}"]'
            )
    cell = await browser_env.page.query_selector(cell_selector)
    if cell is None:
        logger.info("No cell found for room %s,roomid=%s probably doesn't exist.", initial_room_number, room_id)
    else:
        logger.info("Cell found. It means that room %s, roomid=%s is already booked at interval %s", initial_room_number, room_id, interval)

cell 2 JSHandle@node


## Call MetaBookingFormTool 

In [198]:
from src.tools.browser.meta_tools.book_room_form import MetaBookingFormTool

[autoreload of src.tools.browser.meta_tools.fill_book_form failed: Traceback (most recent call last):
  File "/home/olipina/workspace/org-agent/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/olipina/workspace/org-agent/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/olipina/workspace/org-agent/src/tools/browser/meta_tools/fill_book_form.py"

In [199]:
booking_params = {
    "room_number": "2403",
    "start_time": "20:00",
    "end_time": "21:00",
    "building": CONFIG.default_booking_params.building,
    "date": "18-01-2025",
}

In [200]:
result = await MetaBookingFormTool().execute(
    browser_env,
    **booking_params
)

[INFO] src.tools.browser.meta_tools.book_room_form - ********** Navigating to booking page... **********
[INFO] src.tools.browser.click - Attempting to navigate to https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


[INFO] src.tools.browser.click - Navigation successful to https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,
[INFO] src.tools.browser.click - Found texts: ['Бронирование помещений']
[INFO] src.tools.browser.meta_tools.book_room_form - ********** Selecting building... **********
[INFO] src.tools.browser.dropdown - Opening dropdown by selector: .select2-chosen
[INFO] src.tools.browser.dropdown - Retrieving dropdown options
[INFO] src.tools.browser.dropdown - Closing dropdown with Escape key
[INFO] src.tools.browser.dropdown - Successfully retrieved 21 options
[INFO] src.tools.browser.dropdown - Clicking dropdown by selector: .select2-chosen
[INFO] src.tools.browser.dropdown - Selecting option: Аудиторный фонд ИТМО (Кронверкский 49)
[INFO] src.tools.browser.dropdown - Successfully selected 'Аудиторный фонд ИТМО (Кронверкский 49)' from dropdown
[INFO] src.tools.browser.click - Found texts: ['Аудиторный фонд ИТМО (Кронверкский 49)']
[INFO] src.tools.bro

### debug filling form

In [168]:
elements_to_verify = [
    "P4_REQUEST_NAME",  # Event name
    "P4_DATE_START",    # Start time
    "P4_DATE_END",      # End time 
    "P4_PHONE",         # Phone
    "P4_PARTICIPANTS"   # Participants
]


In [163]:
await browser_env.page.fill("#P4_REQUEST_NAME", "Test Event")

In [165]:
input_value = await browser_env.page.input_value("#P4_REQUEST_NAME")
input_value

'Test Event'

In [166]:
await browser_env.page.fill("#P4_DATE_START", "19:00")
await browser_env.page.input_value("#P4_DATE_START")

'19:00'

In [167]:
print("before - ", await browser_env.page.input_value("#P4_DATE_END"))
await browser_env.page.fill("#P4_DATE_END", "21:00") 
print(await browser_env.page.input_value("#P4_DATE_END"))

print("before - ", await browser_env.page.input_value("#P4_PHONE"))
await browser_env.page.fill("#P4_PHONE", "+7 (950) 0121982")
print(await browser_env.page.input_value("#P4_PHONE"))

print("before - ", await browser_env.page.input_value("#P4_PARTICIPANTS"))
await browser_env.page.fill("#P4_PARTICIPANTS", "15")
print(await browser_env.page.input_value("#P4_PARTICIPANTS"))

before -  20:10
21:00
before -  
+7 (950) 0121982
before -  10
15


In [169]:
# Cell 3: Print current form values to verify
for field_id in elements_to_verify:
    value = await browser_env.page.evaluate(f'document.getElementById("{field_id}").value')
    print(f"{field_id}: {value}")

P4_REQUEST_NAME: Test Event
P4_DATE_START: 19:00
P4_DATE_END: 21:00
P4_PHONE: +7 (950) 0121982
P4_PARTICIPANTS: 15


In [173]:
button = await browser_env.page.query_selector("#createProjectButton")
await button.click()

### MetaFillBookingFormTool

In [201]:
from src.tools.browser.meta_tools.fill_book_form import MetaFillBookingFormTool

In [203]:
form_params = {
    "event_name": "Test event Jupyter",
    "start_time": "19:00",
    "end_time": "21:00",
    "phone": "+7 (950) 0121982",
    "participants": 16,
    "save_as_draft": True
}

In [204]:
result = await MetaFillBookingFormTool().execute(
    browser_env,
    **form_params
)

[INFO] src.tools.browser.meta_tools.fill_book_form - All required form elements found
[INFO] src.tools.browser.click - Attempting to fill 'Test event Jupyter' into input #P4_REQUEST_NAME
[INFO] src.tools.browser.click - Successfully filled 'Test event Jupyter' into #P4_REQUEST_NAME
[INFO] src.tools.browser.click - Attempting to fill '19:00' into input #P4_DATE_START
[INFO] src.tools.browser.click - Successfully filled '19:00' into #P4_DATE_START
[INFO] src.tools.browser.click - Attempting to fill '21:00' into input #P4_DATE_END


[INFO] src.tools.browser.click - Successfully filled '21:00' into #P4_DATE_END
[INFO] src.tools.browser.click - Attempting to fill '+7 (950) 0121982' into input #P4_PHONE
[INFO] src.tools.browser.click - Successfully filled '+7 (950) 0121982' into #P4_PHONE
[INFO] src.tools.browser.click - Attempting to fill '16' into input #P4_PARTICIPANTS
[INFO] src.tools.browser.click - Successfully filled '16' into #P4_PARTICIPANTS
[INFO] src.tools.browser.meta_tools.fill_book_form - Form saved as draft successfully


In [205]:
result['meta']

ToolResponse(success=True, error=None, meta={'status': 'draft', 'event_name': 'Test event Jupyter', 'message': 'Form saved as draft'})

In [197]:
"Проект успешно создан." in await browser_env.page.content()

True

In [190]:
print(await browser_env.page.content())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd"><html class="gt-ie8 gt-ie9 not-ie pxajs" lang="ru" xmlns="http://www.w3.org/1999/xhtml" xmlns:htmldb="http://htmldb.oracle.com" xmlns:apex="http://apex.oracle.com"><!--<![endif]--><head>
  <meta charset="utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
  <meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no, minimum-scale=1.0, maximum-scale=1.0">
  <meta name="theme-color" content="#1946ba">
  <meta name="yandex-tableau-widget" content="color=#1946ba">
  <title>Заявка - ИСУ ИТМО [Бронирование помещений]</title>
  <link rel="icon" href="/i/libraries/frontend/misc/favicon.ico" type="image/x-icon">
  <link rel="shortcut icon" href="/i/libraries/frontend/misc/favicon.ico" type="image/x-icon">
  <!-- Legacy -->
  <script type="text/javascript">
var apex_img_dir = "/i/", htmldb_Img_Dir = apex_img_dir;
</script>
<script src="/i/l

# ScenarioSteps

In [5]:
from src.scenarios.booking.login_step import LoginStep
from src.tools.browser.environment import BrowserEnvironment

[INFO] httpx - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
[INFO] config - Configuration loaded successfully from /home/olipina/workspace/org-agent/config.yaml


In [6]:
browser_env = BrowserEnvironment()
await browser_env.initialize()

[INFO] src.tools.browser.environment - Starting new browser environment


In [7]:
login_step = LoginStep()

In [9]:
await login_step.execute(env=browser_env)

[INFO] src.scenarios.booking_steps - Starting login step execution
[INFO] src.scenarios.booking_steps - Navigating to login page: https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688
[INFO] src.tools.browser.click - Attempting to navigate to https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688


[INFO] src.tools.browser.click - Navigation successful to https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688
[INFO] src.tools.browser.click - Found texts: ['Имя пользователя или E-mail']
[INFO] src.scenarios.booking_steps - Filling username field
[INFO] src.tools.browser.click - Attempting to fill '311408' into input #username
[INFO] src.tools.browser.click - Successfully filled '311408' into #username
[INFO] src.scenarios.booking_steps - Filling password field
[INFO] src.tools.browser.click - Attempting to fill 'wrong' into input #password
[INFO] src.tools.browser.click - Successfully filled 'wrong' into #password
[INFO] src.scenarios.booking_steps - Clicking login button – logging in with filled values
[INFO] src.tools.browser.click - Attempting to click button with text: Вход
[INFO] src.tools.browser.click - Waiting for navigat

False

In [12]:
print(login_step.get_execution_history())

Step Status: failed
Total executions: 10
Success rate: 80.0%

Execution Records:

--- Sub-command #1 ---
Navigating to login page: https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688: SUCCESS
Tool: NavigateTool
Time: 2025-01-13 14:07:56
Parameters: {'url': 'https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688'}

Action: navigate
URL: https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688

--- Sub-command #2 ---
Filling username field: SUCCESS
Tool: FillTool
Time: 2025-01-13 14:07:56
Parameters: {'selector': '#username', 'value'

In [16]:
await browser_env.describe_state()

''

In [16]:
print(one_text_response)

❌ FAILED
Tool: CheckContentTool
Time: 2025-01-13 10:27:33
Parameters: {'texts': ['Личный кабинет', 'Центр приложений']}
Error: Missing expected texts: ['Личный кабинет', 'Центр приложений']
Execution steps:
  - Found texts: []
  - Missing texts: ['Личный кабинет', 'Центр приложений']
Browser state at failure:
  URL: https://id.itmo.ru/auth/realms/itmo/login-actions/authenticate?execution=7db2448a-e998-4527-8831-184f67f677f9&client_id=isu&tab_id=UDoC3GLgU_E
  Some visible content:
N/A


#### response

In [7]:
async def handle_response(response):
            print(type(response))
            print(f"Response status: {response.status} | URL: {response.url}")
            print(response.status)
            if response.status in [200, 302]:
                print(f"Successful response received with status: {response.status}")
            else:
                print(f"Received unexpected status code: {response.status}")

        # Define a function to handle the request event
async def handle_request(request):
            print(f"Request made to: {request.url} | Method: {request.method}")

In [8]:
page = browser_env.page

In [9]:
page.on('response', handle_response)

In [ ]:
await page.goto('https://example.com')

<class 'playwright.async_api._generated.Response'>
Response status: 200 | URL: https://example.com/
200
Successful response received with status: 200


<Response url='https://example.com/' request=<Request url='https://example.com/' method='GET'>>

# Scenario calls llm

In [6]:
from src.scenarios.booking.booking import BookingScenario
from src.message_broker import MessageBroker

In [7]:
message_broker = MessageBroker()

[INFO] src.message_broker - Initialized MessageBroker with Redis at redis://localhost:6379


In [10]:
booking_scenario = BookingScenario(message_broker=message_broker)

[INFO] llm_interface - Initializing litellm interface


In [12]:
random_json = {
    "analysis": "The login process failed at the verification step. The expected texts 'Личный кабинет' and 'Центр приложений' were not found, indicating a potential issue with the login credentials, particularly the password which was 'wrong_password'.",
    "next_action": {
        "tool_name": "CallHumanTool",
        "params": {
            "question_to_human": "The login attempt failed due to incorrect password. Please provide the correct password.",
            "options": []
        }
    }
}


In [9]:
await booking_scenario.execute(command="забронируй комнату 2403 на 18 января 2025 с 10 до 13")

[INFO] src.tools.browser.environment - Starting new browser environment


[INFO] scenarios.base - Starting scenario BookingScenario execution for command: забронируй комнату 2403 на 18 января 2025 с 10 до 13
[INFO] llm_interface - Calling model with prompt (300 symbols):
Parse the booking command and extract parameters.
Current date: {current_date}
Command: {user_command}

Return a JSON with this structure:
{{
    "room_count": int,
    "date": "DD-MM-YYYY" | null, # You should calculate this based on the current date
    "start_time": "HH:MM" | null,
    "end_time"
13:41:18 - LiteLLM:INFO: utils.py:2757 - 
LiteLLM completion() model= mistral-large-latest; provider = mistral
[INFO] LiteLLM - 
LiteLLM completion() model= mistral-large-latest; provider = mistral
[INFO] httpx - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
13:41:20 - LiteLLM:INFO: utils.py:917 - Wrapper: Completed Call, calling success_handler
[INFO] LiteLLM - Wrapper: Completed Call, calling success_handler
[INFO] llm_interface - Total cost 0.000986
[INFO] src

CancelledError: 

In [8]:
print(booking_scenario.steps[booking_scenario.context.current_step_index-1].get_execution_history())

Step Status: failed
Total executions: 7
Success rate: 71.4%

Execution Records:

--- Sub-command #1 ---
Navigating to login page: https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688: SUCCESS
Tool: NavigateTool
Time: 2025-01-13 20:51:41
Parameters: {'url': 'https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688'}

URL: https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688

--- Sub-command #2 ---
Filling username field: SUCCESS
Tool: FillTool
Time: 2025-01-13 20:51:41
Parameters: {'selector': '#username', 'value': '[REDACTED]'}

U

# Redis брокер – сообщения

In [12]:
import asyncio
from src.message_broker import MessageBroker

In [13]:
broker = MessageBroker()
await broker.initialize()

[INFO] src.message_broker - Initialized MessageBroker with Redis at redis://localhost:6379
[INFO] src.message_broker - Created consumer group agent_group for stream human_requests
[INFO] src.message_broker - Created consumer group agent_group for stream human_responses


In [14]:
request_id = await broker.send_request(
            scenario_id="test_scenario",
            question="Is this working?",
            options=["Yes", "No"]
        )

[INFO] src.message_broker - Sent human request de6f8cf5-d233-42a6-b408-22d854e467cd for scenario test_scenario: Is this working?


In [15]:
requests = await broker.check_requests()
for request in requests:
            print(f"Found request: {request}")

Found request: request_id='de6f8cf5-d233-42a6-b408-22d854e467cd' scenario_id='test_scenario' question='Is this working?' options=['Yes', 'No'] timestamp=datetime.datetime(2025, 1, 14, 11, 39, 41, 388587) context=None


In [16]:
await broker.send_response(request.request_id, "Yes")

[INFO] src.message_broker - Sent response for request de6f8cf5-d233-42a6-b408-22d854e467cd: Yes


In [17]:
response = await broker.get_response(request_id)
print(f"Got response: {response}")

[INFO] src.message_broker - Received response for request de6f8cf5-d233-42a6-b408-22d854e467cd: Yes


Got response: Yes


In [ ]:


async def main():
    # Initialize broker
    broker = MessageBroker()
    await broker.initialize()
    
    try:
        # Send a test request
        request_id = await broker.send_request(
            scenario_id="test_scenario",
            question="Is this working?",
            options=["Yes", "No"]
        )
        
        print(f"Sent request {request_id}")
        
        # Check for requests (simulating the human side)
        requests = await broker.check_requests()
        for request in requests:
            print(f"Found request: {request}")
            
            # Simulate human responding
            await broker.send_response(request.request_id, "Yes")
            
        # Get the response (back to agent side)
        response = await broker.get_response(request_id)
        print(f"Got response: {response}")
        
    finally:
        await broker.cleanup()

if __name__ == "__main__":
    asyncio.run(main())

# RoomBooking Step

In [5]:
from src.scenarios.booking.login_step import LoginStep
from src.scenarios.booking.navigate_step import NavigateToBookingStep
from src.tools.browser.environment import BrowserEnvironment
from src.scenarios.booking.filling_form_step import RoomBookingStep

[INFO] httpx - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
[INFO] config - Configuration loaded successfully from /home/olipina/workspace/org-agent/config.yaml


In [6]:
browser_env = BrowserEnvironment()
await browser_env.initialize()

[INFO] src.tools.browser.environment - Browser environment initialized


In [7]:
login_step = LoginStep()
result_logging = await login_step.execute(env=browser_env)

[INFO] src.scenarios.booking.login_step - Starting login step execution
[INFO] src.scenarios.booking.login_step - Navigating to login page: https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688
[INFO] src.tools.browser.click - Attempting to navigate to https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688
[INFO] src.tools.browser.click - Navigation successful to https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688
[INFO] src.tools.browser.click - Found texts: ['Имя пользователя или E-mail']
[INFO] src.scenarios.booking.login_ste

In [9]:
await NavigateToBookingStep().execute(env=browser_env)

[INFO] src.scenarios.booking.navigate_step - Starting navigation to booking page
[INFO] src.scenarios.booking.navigate_step - Navigating to booking page: https://isu.ifmo.ru/pls/apex/f?p=2431:1:102628015887802
[INFO] src.tools.browser.click - Attempting to navigate to https://isu.ifmo.ru/pls/apex/f?p=2431:1:102628015887802


[INFO] src.tools.browser.click - Navigation successful to https://isu.ifmo.ru/pls/apex/f?p=2431:1:102628015887802
[INFO] src.scenarios.booking.navigate_step - Clicking 'Create booking' button
[INFO] src.tools.browser.click - Attempting to click button with text: Создать заявку
[INFO] src.tools.browser.click - Waiting for navigation to complete
[INFO] src.tools.browser.click - Click successful on text='Создать заявку'. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:114999787540937::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,
[INFO] src.scenarios.booking.navigate_step - Verifying successful navigation to booking page
[INFO] src.tools.browser.click - Found texts: ['Карточка заявки', 'Заявка', 'Картотека заявок', 'Проект заявки']
[INFO] src.scenarios.booking.navigate_step - Navigation successful - found expected page elements


True

In [19]:
from datetime import time, date
meta = {
    "start_time": time(20, 0),
    "end_time": time(21, 0),
    "building": CONFIG.default_booking_params.building,
    "date": date(2025, 1, 23)
}
room_booking_step = RoomBookingStep(room_number="1405", meta=meta)

In [20]:
await room_booking_step.execute(env=browser_env)

[INFO] src.scenarios.booking.filling_form_step - Booking room 1405
[INFO] src.tools.browser.meta_tools.book_room_form - ********** Navigating to booking page... **********
[INFO] src.tools.browser.click - Attempting to navigate to https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


[INFO] src.tools.browser.click - Navigation successful to https://isu.ifmo.ru/pls/apex/f?p=2431:4:112030779307075::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,
[INFO] src.tools.browser.click - Found texts: ['Бронирование помещений']
[INFO] src.tools.browser.meta_tools.book_room_form - ********** Selecting building... **********
[INFO] src.tools.browser.dropdown - Opening dropdown by selector: .select2-chosen
[INFO] src.tools.browser.dropdown - Retrieving dropdown options
[INFO] src.tools.browser.dropdown - Closing dropdown with Escape key
[INFO] src.tools.browser.dropdown - Successfully retrieved 21 options
[INFO] src.tools.browser.dropdown - Clicking dropdown by selector: .select2-chosen
[INFO] src.tools.browser.dropdown - Selecting option: Аудиторный фонд ИТМО (Кронверкский 49)
[INFO] src.tools.browser.dropdown - Successfully selected 'Аудиторный фонд ИТМО (Кронверкский 49)' from dropdown
[INFO] src.tools.browser.click - Found texts: ['Аудиторный фонд ИТМО (Кронверкский 49)']
[INFO] src.tools.bro

False

In [21]:
input_value = await browser_env.page.input_value("#P4_DATE")
input_value

'23.01.2025'